In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HARLEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
noticias = pd.read_csv('C:/Users/HARLEN/Cientista de dados/TCC/Fakeok/fakeapp3/news.csv')

In [4]:
noticias.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
# Verifica-se a falta de valores no datates
noticias.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [6]:
# Unindo as manchetes com as notícias
noticias['content'] = noticias['title']+' '+noticias['text']

In [7]:
print(noticias['content'])

0       You Can Smell Hillary’s Fear Daniel Greenfield...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathy U....
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [8]:
# separanto o label do dataframe
X = noticias.drop(columns='label', axis=1)
Y = noticias['label']

In [9]:
print(X)
print(Y)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  \
0     Daniel Greenfield, a Shillman Journalism Fello...   
1     Google Pinterest Digg L

In [10]:
#Stemização, aqui vamos reduzir as palavras ao sua raiz(CASinha, CASona, CASebre = CASa)
port_stem = PorterStemmer()
corpus = []

In [11]:
for i in range(0, len(X)):
    stemmed_content = re.sub('[^a-zA-Z]',' ',noticias['content'][i])
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    corpus.append(stemmed_content)

In [12]:
print(noticias['content'])

0       You Can Smell Hillary’s Fear Daniel Greenfield...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathy U....
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [13]:
#separando as noticias e seus valores falso ou positivo, 1 = fake e 0 = verdadeira
X = noticias['content'].values
Y = noticias['label'].values

In [14]:
print(X)

['You Can Smell Hillary’s Fear Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership 

In [15]:
print(Y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [16]:
Y.shape

(6335,)

In [17]:
vectorizer = TfidfVectorizer(max_features=61713, ngram_range=(1, 3))
vectorizer.fit(X)

X1 = vectorizer.transform(X)

In [18]:
print(X)

['You Can Smell Hillary’s Fear Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership 

In [19]:
#Vamos dividir o dataset em train e test
X_train, X_test, Y_train, Y_test = train_test_split(X1, Y, test_size = 0.2, stratify=Y, random_state=2)

In [20]:
#o modelo de previsão utilizado será o LogisticRegression
model = LogisticRegression()

In [21]:
model.fit(X_train, Y_train)

LogisticRegression()

In [22]:
X_train

<5068x61713 sparse matrix of type '<class 'numpy.float64'>'
	with 3565446 stored elements in Compressed Sparse Row format>

In [23]:
# Vamos observar a acurácia do modelo
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [24]:
#98% de acurácia, perfeito
print('Pontuação da acurácia no dataset de treino : ', training_data_accuracy)

Pontuação da acurácia no dataset de treino :  0.9593528018942383


In [25]:
# agora a acurácia no data set de test
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [26]:
#97% o que indica que o modelo está funcionando muito bem
print('Pontuação da acurácia no dataset de teste : ', test_data_accuracy)

Pontuação da acurácia no dataset de teste :  0.9187056037884768


In [27]:

#Modelo 2
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=3)

In [28]:
x_prediction = neigh.predict(X_train)
training_data_accuracy = accuracy_score(x_prediction, Y_train)
print('Pontuação da acurácia no dataset de treino : ', training_data_accuracy)

Pontuação da acurácia no dataset de treino :  0.9161404893449092


In [29]:
#model3
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, Y_train)



SVC()

In [30]:
x_prediction = clf.predict(X_train)
training_data_accuracy = accuracy_score(x_prediction, Y_train)
print('Pontuação da acurácia no dataset de treino : ', training_data_accuracy)

Pontuação da acurácia no dataset de treino :  0.9986187845303868


In [31]:
#Dentro do colchete coloca-se o id da notícia 2.
X_new = X_test[2]

prediction = model.predict(X_new)
print(prediction)



['REAL']


In [32]:
#Aqui temos a confirmação que ela é FAKE
print(Y_test[2])

FAKE


In [33]:
import pickle 
pickle.dump(model, open('model_fakenews.pkl', 'wb'))
pickle.dump(vectorizer, open('tfid.pkl', 'wb'))

In [34]:
tra_model = pickle.load(open('model_fakenews.pkl', 'rb'))
tra_vect  = pickle.load(open('tfid.pkl', 'rb'))
va_pkl = tra_vect.transform([stemmed_content]).toarray()
tra_model.predict(va_pkl)

array(['REAL'], dtype=object)

In [ ]:
#O modelo utilizado para ser salvo, foi o SVC, porque foi o que apresentou melhor acurácia